In [1]:
import sys
print(sys.executable)

import os
print(os.getcwd().replace('object_detection/fish_detection',''))
sys.path.append(os.getcwd().replace('object_detection/fish_detection',''))
sys.path.append(os.getcwd().replace('/fish_detection',''))
print(sys.path)

/anaconda/envs/py35/bin/python
/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/
['', '/opt/caffe/python', '/opt/caffe2/build', '/data/home/azfishvm/notebooks/FindingFishwithTensorflowObjectdetectionAPI/models/research/object_detection/fish_detection', '/anaconda/envs/py35/lib/python35.zip', '/anaconda/envs/py35/lib/python3.5', '/anaconda/envs/py35/lib/python3.5/plat-linux', '/anaconda/envs/py35/lib/python3.5/lib-dynload', '/home/azfishvm/.local/lib/python3.5/site-packages', '/anaconda/envs/py35/lib/python3.5/site-packages', '/anaconda/envs/py35/lib/python3.5/site-packages/dlib-19.15.0-py3.5-linux-x86_64.egg', '/anaconda/envs/py35/lib/python3.5/site-packages/xgboost-0.80-py3.5.egg', '/anaconda/envs/py35/lib/python3.5/site-packages/mxnet-1.3.0-py3.5.egg', '/anaconda/envs/py35/lib/python3.5/site-packages/graphviz-0.8.4-py3.5.egg', '/anaconda/envs/py35/lib/python3.5/site-packages/requests-2.18.4-py3.5.egg', '/anaconda/envs/py35/lib/python3.5/site-p

# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [2]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')


/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Env setup

In [3]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [4]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/anaconda/envs/py35/lib/python3.5/site-packages/object_detection-0.1-py3.5.egg/object_detection/utils/visualization_utils.py:27: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/py35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/envs/py35/lib/python3.5/site-packages/ipykern

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [5]:
#ssd_mobilenet
FINE_TUNED_MODEL_DIR='fishdata/fine_tuned_trained_models/fish_ssd_mobilenet_v1_coco'

#FasterRccn
#FINE_TUNED_MODEL_DIR='fishdata/fine_tuned_trained_models/fish_faster_rcnn_resnet50_coco'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = FINE_TUNED_MODEL_DIR + '/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'fishdata/Videos/video1_output/pascal_label_map.pbtxt'

## Load a (frozen) Tensorflow model into memory.

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [7]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [8]:
def load_image_into_numpy_array(image):
    print(image.size)
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [9]:
path = 'fishdata/test_Frames'
TEST_IMAGE_PATHS=[]
for i in range(0,26500,250):
    TEST_IMAGE_PATHS.append(os.path.join(path,'frame_%d.jpg' %i) )


# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [10]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [11]:
#ssd_mobilenet
resultpath = 'fishdata/test_Frames_ssd_mobilenet_result'

#FasterRccn
#resultpath = 'fishdata/test_Frames_fastrcnn_result'
!  mkdir -p {resultpath}


In [12]:
import os


for image_path in TEST_IMAGE_PATHS:
    print('image_path:',image_path)
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
    head, tail = os.path.split(image_path)
    resul_image_path=os.path.join(resultpath,tail)
    print('resul_image_path:',resul_image_path)
    resultImage = Image.fromarray(image_np)
    resultImage.save(resul_image_path)
    
  #plt.figure(figsize=IMAGE_SIZE)
  #plt.imshow(image_np)

image_path: fishdata/test_Frames/frame_0.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_0.jpg
image_path: fishdata/test_Frames/frame_250.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_250.jpg
image_path: fishdata/test_Frames/frame_500.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_500.jpg
image_path: fishdata/test_Frames/frame_750.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_750.jpg
image_path: fishdata/test_Frames/frame_1000.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_1000.jpg
image_path: fishdata/test_Frames/frame_1250.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_1250.jpg
image_path: fishdata/test_Frames/frame_1500.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_1500.jpg
image_path: fishdata/test_Frames/frame_1750.jpg
(1920, 1080)

resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_15000.jpg
image_path: fishdata/test_Frames/frame_15250.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_15250.jpg
image_path: fishdata/test_Frames/frame_15500.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_15500.jpg
image_path: fishdata/test_Frames/frame_15750.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_15750.jpg
image_path: fishdata/test_Frames/frame_16000.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_16000.jpg
image_path: fishdata/test_Frames/frame_16250.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_16250.jpg
image_path: fishdata/test_Frames/frame_16500.jpg
(1920, 1080)
resul_image_path: fishdata/test_Frames_ssd_mobilenet_result/frame_16500.jpg
image_path: fishdata/test_Frames/frame_16750.jpg
(1920, 1080)
resul_image_path: fishdata/test_Fr

In [13]:
! ls {resultpath}

frame_0.jpg	 frame_15750.jpg  frame_22000.jpg  frame_4250.jpg
frame_10000.jpg  frame_16000.jpg  frame_22250.jpg  frame_4500.jpg
frame_1000.jpg	 frame_16250.jpg  frame_22500.jpg  frame_4750.jpg
frame_10250.jpg  frame_16500.jpg  frame_2250.jpg   frame_5000.jpg
frame_10500.jpg  frame_16750.jpg  frame_22750.jpg  frame_500.jpg
frame_10750.jpg  frame_17000.jpg  frame_23000.jpg  frame_5250.jpg
frame_11000.jpg  frame_17250.jpg  frame_23250.jpg  frame_5500.jpg
frame_11250.jpg  frame_17500.jpg  frame_23500.jpg  frame_5750.jpg
frame_11500.jpg  frame_1750.jpg   frame_23750.jpg  frame_6000.jpg
frame_11750.jpg  frame_17750.jpg  frame_24000.jpg  frame_6250.jpg
frame_12000.jpg  frame_18000.jpg  frame_24250.jpg  frame_6500.jpg
frame_12250.jpg  frame_18250.jpg  frame_24500.jpg  frame_6750.jpg
frame_12500.jpg  frame_18500.jpg  frame_24750.jpg  frame_7000.jpg
frame_1250.jpg	 frame_18750.jpg  frame_25000.jpg  frame_7250.jpg
frame_12750.jpg  frame_19000.jpg  frame_2500.jpg   frame_7500.jpg
frame_13000.jpg  